# Supervised Model- Random Forest 


In [15]:
# Install some libraries 

# pip install numpy  
# !pip install scikit-learn

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score
from IPython.display import clear_output


In [17]:
import pandas as pd 
import numpy as np 

In [18]:
df = pd.read_csv('imputed_dataset.csv')
df.head()

,rating,ranking,year,position,height,weight,latitude,longitude,state_province,stars,committed_to,is_drafted,conference,latitude_school,longitude_school,distance_miles,side_of_ball,position_group
0,0.9992,1.0,2015,DT,74.0,313.0,31.578206,-84.155681,GA,5,Georgia,0.0,SEC,33.94982,-83.373381,170.049603,defense,d_line
1,0.9991,1.0,2015,DT,74.5,313.0,31.578206,-84.155681,GA,5,Georgia,0.0,SEC,33.94982,-83.373381,170.049603,defense,d_line
2,0.9879,27.0,2015,ATH,72.0,168.0,33.173177,-84.914936,GA,5,Georgia,1.0,SEC,33.94982,-83.373381,103.714733,athlete,athlete
3,0.9769,48.0,2015,OLB,74.0,207.0,32.305158,-84.027407,GA,4,Georgia,1.0,SEC,33.94982,-83.373381,119.771399,defense,linebacker
4,0.9680,65.0,2015,SDE,76.0,265.0,33.853270,-84.220073,GA,4,Georgia,0.0,SEC,33.94982,-83.373381,49.011958,defense,d_line


In [19]:
df.dtypes 

rating              float64
ranking             float64
year                  int64
position             object
height              float64
weight              float64
latitude            float64
longitude           float64
state_province       object
stars                 int64
committed_to         object
is_drafted          float64
conference           object
latitude_school     float64
longitude_school    float64
distance_miles      float64
side_of_ball         object
position_group       object
dtype: object

In [20]:
# Define numerical and categorical values
numerical_features = ['rating', 'ranking', 'height', 'weight', 'distance_miles', 'stars']
categorical_features = ['side_of_ball', 'position_group']
target = ['is_drafted']


In [21]:
# Features and target
X = df[numerical_features+categorical_features]
y = df[target]


In [22]:
# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


# Create a Random Forest model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])


In [23]:
# Define the parameter grid for GridSearchCV
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_features': ['log2'],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model,
                           param_grid=param_grid,
                           cv=5,  # Number of cross-validation folds
                           scoring='roc_auc',  # Scoring metric
                           n_jobs=-1,  # Use all available cores
                           verbose=2)  # Increase verbosity


In [24]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [25]:
%%time 
# Train the model with GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best model from grid search
best_model = grid_search.best_estimator_

clear_output()  # Clear the output


CPU times: user 3.47 s, sys: 588 ms, total: 4.06 s
Wall time: 5min 36s


In [26]:
# Train the model
best_model.fit(X_train, y_train)

/Users/andreeaserban/anaconda3/envs/MilestoneI/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['rating', 'ranking',
                                                   'height', 'weight',
                                                   'distance_miles', 'stars']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['side_of_ball',
                                                   'position_group'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=10, max_features='log2',
                                        min_samples_split=5,
                                        random_state=42))])

In [27]:
# Make predictions

y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class


In [28]:
# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Optionally: Print feature importances
importances = best_model.named_steps['classifier'].feature_importances_
print("Feature Importances:")
print(importances)

Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97      6485
         1.0       0.12      0.00      0.01       351

    accuracy                           0.95      6836
   macro avg       0.54      0.50      0.49      6836
weighted avg       0.91      0.95      0.92      6836

Feature Importances:
[2.42139690e-01 2.67053006e-01 8.57442651e-02 1.32039703e-01
 1.47089224e-01 6.48385469e-02 3.77128729e-03 4.88700564e-03
 6.43537860e-03 5.94855206e-04 2.62857890e-07 3.12178836e-03
 6.28728823e-03 4.12542319e-03 9.03147027e-03 4.79109576e-03
 7.42022353e-03 4.56631395e-03 5.44300295e-03 6.20030234e-04
 1.39508524e-07]


In [29]:
# Calculate ROC AUC score
roc_auc = roc_auc_score(y_test, y_prob)
print(f"ROC AUC Score: {roc_auc:.4f}")

# Get feature importances
importances = best_model.named_steps['classifier'].feature_importances_

# Get feature names after preprocessing
feature_names = (best_model.named_steps['preprocessor']
                  .transformers_[1][1]
                  .named_steps['onehot']
                  .get_feature_names_out(categorical_features))
all_feature_names = numerical_features + feature_names.tolist()

# Combine feature names with their importances
feature_importances = pd.DataFrame({
    'Feature': all_feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print("Feature Importances:")
print(feature_importances)

ROC AUC Score: 0.6937
Feature Importances:
                        Feature    Importance
1                       ranking  2.670530e-01
0                        rating  2.421397e-01
4                distance_miles  1.470892e-01
3                        weight  1.320397e-01
2                        height  8.574427e-02
5                         stars  6.483855e-02
14    position_group_linebacker  9.031470e-03
16  position_group_pass_catcher  7.420224e-03
8          side_of_ball_offense  6.435379e-03
12   position_group_d_backfield  6.287288e-03
18  position_group_running_back  5.443003e-03
7          side_of_ball_defense  4.887006e-03
15        position_group_o_line  4.791096e-03
17            position_group_qb  4.566314e-03
13        position_group_d_line  4.125423e-03
6          side_of_ball_athlete  3.771287e-03
11       position_group_athlete  3.121788e-03
19       position_group_special  6.200302e-04
9          side_of_ball_special  5.948552e-04
10         side_of_ball_unknown  2.62